In [1]:
import logging
import sys
import os

# LlamaIndex imports
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.llms.bedrock import Bedrock
from llama_index.core.node_parser import SentenceSplitter
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
# Make sure your AWS credentials are configured (e.g., via AWS CLI, environment variables, or IAM roles)
import boto3
from botocore.exceptions import BotoCoreError, ClientError


#  Set up AWS Bedrock LLM and Embedding Model

You'll need to specify your AWS region and the model IDs

In [ ]:
# cluade LLM model
{
  "modelId": "anthropic.claude-3-5-sonnet-20240620-v1:0",
  "contentType": "application/json",
  "accept": "application/json",
  "body": {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1000,
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "image",
            "source": {
              "type": "base64",
              "media_type": "image/jpeg",
              "data": "iVBORw..."
            }
          },
          {
            "type": "text",
            "text": "What's in this image?"
          }
        ]
      }
    ]
  }
}

In [ ]:
# amazon titan embedding model
{
 "modelId": "amazon.titan-embed-text-v2:0",
 "contentType": "application/json",
 "accept": "*/*",
 "body": "{\"inputText\":\"this is where you place your input text\", \"dimensions\": 512, \"normalize\": true}"
}

In [2]:
import boto3
import json
import os
import requests
import warnings
warnings.filterwarnings('ignore')
from botocore.exceptions import BotoCoreError, ClientError

from dotenv import load_dotenv, find_dotenv

class ClaudeModel:
    def __init__(self):
        dotenv_path = find_dotenv()
        if not dotenv_path:
            raise FileNotFoundError("Could not find .env file")
        load_dotenv(dotenv_path)
        
        self.aws_access_key_id = os.getenv("aws_access_key_id")
        self.aws_secret_access_key = os.getenv("aws_secret_access_key")
        self.aws_session_token = os.getenv("aws_session_token")
        
        if not all([self.aws_access_key_id, self.aws_secret_access_key, self.aws_session_token]):
            raise ValueError("One or more AWS credentials are missing in the .env file")

        try:
            self.brt = boto3.client(
                service_name='bedrock-runtime',
                region_name='eu-central-1',
                aws_access_key_id=self.aws_access_key_id,
                aws_secret_access_key=self.aws_secret_access_key,
                aws_session_token=self.aws_session_token
            )
        except (BotoCoreError, ClientError) as error:
            raise ConnectionError(f"Failed to create boto3 client: {error}")

        self.modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
        self.accept = 'application/json'
        self.contentType = 'application/json'

    def generate_response(self, prompt_text, temperature, ai_role):
        body = json.dumps({
            "messages": [
                {"role": "assistant", "content": ai_role},
                {"role": "user", "content": [{'type': 'text', "text": prompt_text}]}
            ],
            "max_tokens": 5000,
            "temperature": temperature,
            "top_p": 0.9,
            "anthropic_version": "bedrock-2023-05-31"
        })

        try:
            response = self.brt.invoke_model(body=body, modelId=self.modelId, accept=self.accept, contentType=self.contentType)
            response_body = json.loads(response.get('body').read())
        except (BotoCoreError, ClientError) as error:
            raise RuntimeError(f"Failed to invoke model: {error}")
        except json.JSONDecodeError as error:
            raise ValueError(f"Failed to parse response body: {error}")

        try:
            text = response_body['content'][0]['text']
            usage = response_body['usage']
            prompt_tokens = usage['input_tokens']
            response_tokens = usage['output_tokens']
            total_tokens = prompt_tokens + response_tokens
        except KeyError as error:
            raise KeyError(f"Missing expected key in response body: {error}")

        return prompt_tokens, response_tokens, total_tokens, text

In [4]:
model = ClaudeModel()

model.generate_response('Give me beef stew simple recipe?', 0.1, 'You are a helpful assistant.')

(36,
 384,
 420,
 "Here is a simple recipe for beef stew:\n\nIngredients:\n- 1 lb beef stew meat, cut into 1-inch cubes\n- 3 tablespoons all-purpose flour\n- 2 tablespoons olive oil\n- 1 onion, diced\n- 3 carrots, sliced\n- 3 potatoes, diced \n- 3 cups beef broth\n- 1 bay leaf\n- 1 teaspoon dried thyme\n- Salt and pepper to taste\n\nInstructions:\n\n1. Pat the beef dry and toss it with the flour until coated. Shake off any excess flour.\n\n2. Heat the olive oil in a large pot or dutch oven over medium-high heat. Add the beef in a single layer and brown on all sides, about 2-3 minutes per side. Transfer browned beef to a plate. \n\n3. Add the onions to the pot and cook for 2-3 minutes until translucent. Add the carrots and potatoes and cook for 2 more minutes.\n\n4. Pour in the beef broth and use a wooden spoon to scrape up any browned bits from the bottom of the pot. Add the bay leaf, thyme, salt and pepper. \n\n5. Return the beef and any juices to the pot. Bring to a boil, then reduce

In [ ]:
{
 "modelId": "amazon.titan-embed-text-v2:0",
 "contentType": "application/json",
 "accept": "*/*",
 "body": "{\"inputText\":\"this is where you place your input text\", \"dimensions\": 512, \"normalize\": true}"
}

In [5]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
"""
Shows how to generate an embedding with the Amazon Titan Embeddings G1 - Text model (on demand).
"""

import json
import logging
import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
import warnings
warnings.filterwarnings("ignore")

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def generate_embedding(model_id, body):
    """
    Generate an embedding with the vector representation of a text input using Amazon Titan Embeddings G1 - Text on demand.
    Args:
        model_id (str): The model ID to use.
        body (str) : The request body to use.
    Returns:
        response (JSON): The embedding created by the model and the number of input tokens.
    """
    logger.info("Generating an embedding with Amazon Titan Embeddings G1 - Text model %s", model_id)

    bedrock = boto3.client(
                service_name='bedrock-runtime',
                region_name='eu-central-1',
                aws_access_key_id=os.environ.get("aws_access_key_id"),
                aws_secret_access_key=os.environ.get("aws_secret_access_key"),
                aws_session_token=os.environ.get("aws_session_token")
            )

    accept = "application/json"
    content_type = "application/json"
    response = bedrock.invoke_model(
                                    body=body, 
                                    modelId=model_id, 
                                    accept=accept, 
                                    contentType=content_type
                                )
    response_body = json.loads(response.get('body').read())

    return response_body

def main():
    """
    Entrypoint for Amazon Titan Embeddings G1 - Text example.
    """

    logging.basicConfig(level=logging.INFO,
                        format="%(levelname)s: %(message)s")

    model_id = "amazon.titan-embed-text-v1"
    input_text = "What are the different services that you offer?"

    # Create request body.
    body = json.dumps({
        "inputText": input_text,
    })
    try:

        response = generate_embedding(model_id, body)

        print(f"Generated an embedding: {response['embedding']}")
        print(f"Input Token count:  {response['inputTextTokenCount']}")

    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        print("A client error occured: " +
              format(message))

    else:
        print(f"Finished generating an embedding with Amazon Titan Embeddings G1 - Text model {model_id}.")

if __name__ == "__main__":
    main()


INFO:__main__:Generating an embedding with Amazon Titan Embeddings G1 - Text model amazon.titan-embed-text-v1
Generating an embedding with Amazon Titan Embeddings G1 - Text model amazon.titan-embed-text-v1
Generated an embedding: [-0.10595703125, -0.2177734375, 0.291015625, -0.1962890625, -0.26953125, 0.3828125, -0.291015625, 7.2479248046875e-05, -0.322265625, 0.2431640625, -0.345703125, 0.01483154296875, 0.00787353515625, -0.185546875, 0.0546875, -0.345703125, 0.09716796875, 0.734375, 0.333984375, 0.039794921875, 0.103515625, 0.2216796875, 0.31640625, -0.6171875, 0.55078125, 0.076171875, 0.4296875, 0.2333984375, 0.357421875, 0.578125, 0.11181640625, -0.2578125, -0.08984375, -0.7578125, -0.19921875, -0.015625, -0.2236328125, 0.146484375, 0.03857421875, -0.1591796875, 0.3515625, 0.392578125, 0.80859375, 0.498046875, 0.92578125, -0.0634765625, -0.041015625, 0.201171875, 0.322265625, 0.10205078125, 0.11572265625, -0.474609375, -0.035400390625, -0.09033203125, 0.255859375, -0.40234375, 0.5

In [14]:
# Configure AWS region (replace with your actual region)
aws_region = "us-east-1" # Or your preferred region with Bedrock access

# Initialize Bedrock LLM (Claude 3.5 Sonnet)
# Ensure you have model access enabled in the Bedrock console for this model.
llm = Bedrock(
    model="anthropic.claude-3-5-sonnet-20240620-v1:0", # Claude 3.5 Sonnet model ID
    region_name=aws_region,
    # You can pass additional model_kwargs if needed, e.g., temperature, max_tokens_to_sample
    # model_kwargs={"temperature": 0.1, "max_tokens_to_sample": 512}
)
# llm = ClaudeModel()
# Initialize Bedrock Embedding Model (e.g., Amazon Titan Embeddings G1 - Text)
# Ensure you have model access enabled in the Bedrock console for this model.
embed_model = BedrockEmbedding(
    model="amazon.titan-embed-text-v2:0",
    region_name=aws_region,
    aws_access_key_id=os.environ.get("aws_access_key_id"),
    aws_secret_access_key=os.environ.get("aws_secret_access_key"),
    aws_session_token=os.environ.get("aws_session_token")
    
)

# Set the global settings for LlamaIndex
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512 # Optional: configure chunk size for document splitting
Settings.chunk_overlap = 20 # Optional: configure chunk overlap

INFO:botocore.credentials:Found credentials in environment variables.
Found credentials in environment variables.


# Load Your Documents

In [8]:
# Define the path to your data directory
data_directory = "../data" # Relative path from notebooks to src/data

# Check if the directory exists and create it if it doesn't
if not os.path.exists(data_directory):
    os.makedirs(data_directory)
    print(f"Created directory: {data_directory}")
    # Add a dummy file for testing if the directory was just created
    with open(os.path.join(data_directory, "sample_document.txt"), "w") as f:
        f.write("This is a sample document for the LlamaIndex RAG system. ")
        f.write("It contains information about AI and machine learning. ")
        f.write("AWS Bedrock provides access to powerful foundation models.")
    print(f"Created sample_document.txt in {data_directory}")

try:
    documents = SimpleDirectoryReader(data_directory).load_data()
    if not documents:
        print(f"No documents found in {data_directory}. Please add some text files.")
    else:
        print(f"Loaded {len(documents)} document(s).")
except Exception as e:
    print(f"Error loading documents: {e}")
    print(f"Please ensure the directory '{data_directory}' exists and contains readable files.")
    documents = [] # Initialize as empty list to prevent further errors

Loaded 295 document(s).


In [9]:
documents

[Document(id_='4948f505-4573-4e7f-9769-c776ac30ef23', embedding=None, metadata={'page_label': '1', 'file_name': 'Currency and Exchanges Manual for Authorised Dealers.pdf', 'file_path': 'c:\\Users\\A238737\\OneDrive - Standard Bank\\Documents\\Africa Regions\\Ad - Hoc\\CIBOps\\ExconAI\\ExconAI-Payment-Instructions\\aws\\src\\rag\\notebooks\\..\\data\\Currency and Exchanges Manual for Authorised Dealers.pdf', 'file_type': 'application/pdf', 'file_size': 2174852, 'creation_date': '2025-05-21', 'last_modified_date': '2025-02-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Currency and Exchanges \nManual for Authorised Dealers \n \n2024-1

# Create the Index

This step will process your documents, create embeddings, and store them in a vector index.

In [ ]:
# create a custom embedding model for AWS Bedrock Titan
# The default LlamaIndex class doesnt work in my case
from llama_index.core.embeddings import BaseEmbedding
from llama_index.core.bridge.pydantic import Field, PrivateAttr
from typing import List, Any, Optional
import asyncio
import boto3
import aioboto3
import json
import os
from botocore.config import Config

class CustomTitanEmbedding(BaseEmbedding):
    """Custom Titan Embedding implementation for AWS Bedrock."""
    
    # Public fields (following BedrockEmbedding pattern)
    model_name: str = Field(default="amazon.titan-embed-text-v1", description="The modelId of the Bedrock model to use.")
    aws_access_key_id: Optional[str] = Field(default=None, description="AWS Access Key ID to use")
    aws_secret_access_key: Optional[str] = Field(default=None, description="AWS Secret Access Key to use")
    aws_session_token: Optional[str] = Field(default=None, description="AWS Session Token to use")
    region_name: Optional[str] = Field(default="eu-central-1", description="AWS region name to use")
    max_retries: int = Field(default=10, description="The maximum number of API retries.", gt=0)
    timeout: float = Field(default=60.0, description="The timeout for the Bedrock API request in seconds")
    
    # Private attributes (following BedrockEmbedding pattern)
    _config: Any = PrivateAttr()
    _client: Any = PrivateAttr()
    _asession: Any = PrivateAttr()
    
    def __init__(
        self,
        model_name: str = "amazon.titan-embed-text-v1",
        aws_access_key_id: Optional[str] = None,
        aws_secret_access_key: Optional[str] = None,
        aws_session_token: Optional[str] = None,
        region_name: str = "eu-central-1",
        max_retries: int = 10,
        timeout: float = 60.0,
        **kwargs: Any,
    ):
        # Initialize the parent class first
        super().__init__(
            model_name=model_name,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            aws_session_token=aws_session_token,
            region_name=region_name,
            max_retries=max_retries,
            timeout=timeout,
            **kwargs
        )
        
        # Set up credentials from environment if not provided
        self.aws_access_key_id = aws_access_key_id or os.environ.get("aws_access_key_id")
        self.aws_secret_access_key = aws_secret_access_key or os.environ.get("aws_secret_access_key")
        self.aws_session_token = aws_session_token or os.environ.get("aws_session_token")
        
        # Initialize client configuration
        self._setup_clients()
    
    def _setup_clients(self):
        """Initialize the Bedrock clients with proper credentials."""
        try:
            # Setup session kwargs
            session_kwargs = {
                "region_name": self.region_name,
                "aws_access_key_id": self.aws_access_key_id,
                "aws_secret_access_key": self.aws_secret_access_key,
                "aws_session_token": self.aws_session_token,
            }
            
            # Remove None values from session_kwargs
            session_kwargs = {k: v for k, v in session_kwargs.items() if v is not None}
            
            # Setup botocore config
            self._config = Config(
                retries={"max_attempts": self.max_retries, "mode": "standard"},
                connect_timeout=self.timeout,
                read_timeout=self.timeout,
            )
            
            # Create sync session and client
            session = boto3.Session(**session_kwargs)
            self._client = session.client("bedrock-runtime", config=self._config)
            
            # Create async session
            self._asession = aioboto3.Session(**session_kwargs)
            
        except ImportError:
            raise ImportError(
                "boto3 and/or aioboto3 package not found, install with 'pip install boto3 aioboto3'"
            )
        except Exception as e:
            raise ConnectionError(f"Failed to setup AWS Bedrock clients: {e}")
    
    def _get_query_embedding(self, query: str) -> List[float]:
        """Get embedding for a query string."""
        return self._get_embedding(query)
    
    async def _aget_query_embedding(self, query: str) -> List[float]:
        """Async version of query embedding."""
        return await self._aget_embedding(query)
    
    def _get_text_embedding(self, text: str) -> List[float]:
        """Get embedding for a text string."""
        return self._get_embedding(text)
    
    async def _aget_text_embedding(self, text: str) -> List[float]:
        """Async version of text embedding."""
        return await self._aget_embedding(text)
    
    def _get_embedding(self, text: str) -> List[float]:
        """Core method to generate embeddings using AWS Bedrock Titan model."""
        try:
            # Prepare the request body (simplified for v1 model)
            if self.model_name == "amazon.titan-embed-text-v1":
                body = json.dumps({"inputText": text})
            else:
                # For v2 model, include additional parameters
                body = json.dumps({
                    "inputText": text,
                    "dimensions": 1024,  # v2 default
                    "normalize": True
                })
            
            # Call Bedrock API
            response = self._client.invoke_model(
                body=body,
                modelId=self.model_name,
                accept="application/json",
                contentType="application/json"
            )
            
            # Parse response
            response_body = json.loads(response.get('body').read())
            embedding = response_body['embedding']
            
            return embedding
            
        except Exception as e:
            print(f"Error generating embedding for text: '{text[:50]}...': {e}")
            # Return a zero vector as fallback based on model type
            fallback_dim = 1536 if "v1" in self.model_name else 1024
            return [0.0] * fallback_dim
    
    async def _aget_embedding(self, text: str) -> List[float]:
        """Async version of embedding generation."""
        try:
            # Prepare the request body
            if self.model_name == "amazon.titan-embed-text-v1":
                body = json.dumps({"inputText": text})
            else:
                body = json.dumps({
                    "inputText": text,
                    "dimensions": 1024,
                    "normalize": True
                })
            
            # Use async client
            async with self._asession.client("bedrock-runtime", config=self._config) as client:
                response = await client.invoke_model(
                    body=body,
                    modelId=self.model_name,
                    accept="application/json",
                    contentType="application/json"
                )
                
                streaming_body = await response.get("body").read()
                response_body = json.loads(streaming_body.decode("utf-8"))
                embedding = response_body['embedding']
                
                return embedding
                
        except Exception as e:
            print(f"Error generating async embedding for text: '{text[:50]}...': {e}")
            # Return a zero vector as fallback
            fallback_dim = 1536 if "v1" in self.model_name else 1024
            return [0.0] * fallback_dim

# Create and test the embedding model with proper error handling
def setup_custom_embedding():
    """Setup function to initialize the custom embedding model with fallback."""
    try:
        print("Initializing Custom Titan Embedding...")
        embed_model = CustomTitanEmbedding()
        
        # Test the embedding
        test_text = "This is a test sentence."
        test_embedding = embed_model._get_embedding(test_text)
        print(f"✅ Custom embedding successful! Dimensions: {len(test_embedding)}")
        
        return embed_model
        
    except Exception as e:
        print(f"❌ Failed to initialize CustomTitanEmbedding: {e}")
        print("🔄 Falling back to standard BedrockEmbedding...")
        
        try:
            from llama_index.embeddings.bedrock import BedrockEmbedding
            embed_model = BedrockEmbedding(
                model_name="amazon.titan-embed-text-v1",
                region_name="eu-central-1",
                aws_access_key_id=os.environ.get("aws_access_key_id"),
                aws_secret_access_key=os.environ.get("aws_secret_access_key"),
                aws_session_token=os.environ.get("aws_session_token")
            )
            print("✅ Fallback embedding model initialized successfully.")
            return embed_model
            
        except Exception as fallback_error:
            print(f"❌ Fallback also failed: {fallback_error}")
            raise RuntimeError("Both custom and fallback embedding models failed to initialize")

# Use the setup function
embed_model = setup_custom_embedding()
Settings.embed_model = embed_model

Initializing Custom Titan Embedding...
✅ Custom embedding successful! Dimensions: 1536


In [21]:
if documents:
    try:
        index = VectorStoreIndex.from_documents(
            documents,
        )
        print("Index created successfully.")
    except Exception as e:
        print(f"Error creating index: {e}")
        print("This might be due to model access permissions. Please check your AWS Bedrock model access.")
        index = None
else:
    print("Skipping index creation as no documents were loaded.")
    index = None

Index created successfully.


# Create a Query Engine

The query engine is used to ask questions against your indexed data.

In [22]:
if index:
    query_engine = index.as_query_engine(
        similarity_top_k=3, # Retrieve top 3 most similar documents
        # You can customize other parameters like response_mode
    )
    print("Query engine created successfully.")
else:
    print("Skipping query engine creation as index is not available.")
    query_engine = None

Query engine created successfully.


# Query Your Data

In [ ]:
if query_engine:
    query_text = "What is AWS Bedrock?" # Replace with your query
    print(f"\nQuerying: {query_text}")
    response = query_engine.query(query_text)
    print("\nResponse:")
    print(response)

    print("\n\nSource Nodes:")
    for node in response.source_nodes:
        print(f"Node ID: {node.node_id}, Score: {node.score:.4f}")
        print(f"Text: {node.text[:150]}...") # Print first 150 chars of source text
        print("-" * 20)
else:
    print("Cannot query as the query engine is not available.")

In [ ]:
def query_documents(user_input, top_k=3):
    """
    Query the indexed documents with user input and return the model's response.
    
    Args:
        user_input (str): The question or query from the user
        top_k (int): Number of top similar documents to retrieve (default: 3)
    
    Returns:
        dict: Dictionary containing the response text, source information, and metadata
    """
    if not query_engine:
        return {
            "error": "Query engine is not available. Please ensure documents are loaded and indexed.",
            "response": None,
            "sources": []
        }
    
    try:
        # Query the engine
        response = query_engine.query(user_input)
        
        # Extract source information
        sources = []
        for node in response.source_nodes:
            sources.append({
                "node_id": node.node_id,
                "score": round(node.score, 4),
                "text_preview": node.text[:150] + "..." if len(node.text) > 150 else node.text,
                "full_text": node.text
            })
        
        return {
            "error": None,
            "response": str(response),
            "sources": sources,
            "query": user_input,
            "num_sources": len(sources)
        }
        
    except Exception as e:
        return {
            "error": f"Error processing query: {str(e)}",
            "response": None,
            "sources": [],
            "query": user_input
        }

In [ ]:
# Example usage:
result = query_documents("What is AWS Bedrock?")
if result["error"]:
    print(f"Error: {result['error']}")
else:
    print(f"Query: {result['query']}")
    print(f"\nResponse:\n{result['response']}")
    print(f"\nFound {result['num_sources']} source(s):")
    for i, source in enumerate(result['sources'], 1):
        print(f"\n{i}. Score: {source['score']}")
        print(f"   Preview: {source['text_preview']}")

Query: What is AWS Bedrock?

Response:
I apologize, but I don't have any information about AWS Bedrock in the given context. The provided context primarily discusses financial regulations, exchange control compliance, and data handling related to cross-border transactions. It does not contain any information about AWS Bedrock or other cloud computing services.

Found 3 source(s):

1. Score: 0.0711
   Preview: Currency and Exchanges Manual for Authorised Dealers J. 
 256 of 295 2/2022 
(D) Offshoring and cloud computing 
 
(i) For the purpose of this subsect...

2. Score: 0.0673
   Preview: These discretionary financial 
services provider s, who elect not to register with the Financial 
Surveillance Department, are treated as intermediari...

3. Score: 0.0647
   Preview: Refer to the pre and post certification 
managerial letter of comfort specimens that are available as 
outlined in (i)(c) above. 
 
(iii)  Annual mana...


In [26]:
def generate_response(user_input):
    """
    Run an example query using the documents and return the result.
    
    Returns:
        dict: Dictionary containing the response text, source information, and metadata
    """
    result = query_documents(user_input)
    if result["error"]:
        print(f"Error: {result['error']}")
    else:
        print(f"Query: {result['query']}")
        print(f"\nResponse:\n{result['response']}")
        print(f"\nFound {result['num_sources']} source(s):")
        for i, source in enumerate(result['sources'], 1):
            print(f"\n{i}. Score: {source['score']}")
            print(f"   Preview: {source['text_preview']}")
    
    return result

In [27]:
generate_response("what is Authorised Dealer Manual")

Query: what is Authorised Dealer Manual

Response:
The Authorised Dealer Manual is a comprehensive document that outlines the permissions, conditions, and limits applicable to foreign exchange transactions that can be undertaken by Authorised Dealers or on behalf of their clients. It also includes details of related administrative responsibilities.

This manual serves as a key reference for Authorised Dealers, providing guidance on how to handle various foreign exchange transactions within the regulatory framework. It contains specific instructions and rules that Authorised Dealers must follow when dealing with foreign currency transactions.

The manual is designed to ensure uniformity of policy and strict adherence to regulations. It is used in conjunction with other documents such as the Regulations, circulars, and other directives to govern foreign exchange activities in South Africa.

It's important to note that Authorised Dealers are expected to apply the contents of this manual s

{'error': None,
 'response': "The Authorised Dealer Manual is a comprehensive document that outlines the permissions, conditions, and limits applicable to foreign exchange transactions that can be undertaken by Authorised Dealers or on behalf of their clients. It also includes details of related administrative responsibilities.\n\nThis manual serves as a key reference for Authorised Dealers, providing guidance on how to handle various foreign exchange transactions within the regulatory framework. It contains specific instructions and rules that Authorised Dealers must follow when dealing with foreign currency transactions.\n\nThe manual is designed to ensure uniformity of policy and strict adherence to regulations. It is used in conjunction with other documents such as the Regulations, circulars, and other directives to govern foreign exchange activities in South Africa.\n\nIt's important to note that Authorised Dealers are expected to apply the contents of this manual strictly and imp

In [28]:
generate_response(" show me the Name of entity - Authorised Dealer")

Query:  show me the Name of entity - Authorised Dealer

Response:
The entity referred to as "Authorised Dealer" appears to be a financial institution or bank that has been authorized to handle certain foreign exchange transactions and travel allowances. These Authorised Dealers have specific responsibilities and permissions outlined in the provided context, such as:

1. Recording personal information when providing travel allowances to individuals who don't have accounts with them.

2. Handling applications for listing South African registered companies on the JSE Limited (excluding banks and bank holding companies).

3. Receiving written confirmation from companies regarding ad hoc services between related parties.

4. Ensuring proper reporting of royalties and fees on the FinSurv Reporting System.

5. Following specific guidelines when dealing with visiting artistes, entertainers, sportspersons, and similar professionals.

These Authorised Dealers seem to play a significant role in m

{'error': None,
 'response': 'The entity referred to as "Authorised Dealer" appears to be a financial institution or bank that has been authorized to handle certain foreign exchange transactions and travel allowances. These Authorised Dealers have specific responsibilities and permissions outlined in the provided context, such as:\n\n1. Recording personal information when providing travel allowances to individuals who don\'t have accounts with them.\n\n2. Handling applications for listing South African registered companies on the JSE Limited (excluding banks and bank holding companies).\n\n3. Receiving written confirmation from companies regarding ad hoc services between related parties.\n\n4. Ensuring proper reporting of royalties and fees on the FinSurv Reporting System.\n\n5. Following specific guidelines when dealing with visiting artistes, entertainers, sportspersons, and similar professionals.\n\nThese Authorised Dealers seem to play a significant role in managing various financi

In [ ]:
generate_response(" list Authorised Dealers")

Query:  list Authorised Dealer

Response:
Here is a list of Authorised Dealers:

1. ABSA Bank Limited
2. Bidvest Bank Limited
3. Citibank N.A., South Africa Branch
4. FirstRand Bank Limited
5. HSBC Bank plc – Johannesburg Branch
6. Investec Bank Limited
7. Mercantile Bank Limited
8. Nedbank Limited
9. Sasfin Bank Limited
10. Société Générale Johannesburg Branch
11. Standard Chartered Bank – Johannesburg Branch
12. The Standard Bank of South Africa Limited

Each of these institutions has specific authorized branches or operational centers listed as well.

Found 3 source(s):

1. Score: 0.504
   Preview: Currency and Exchanges Manual for Authorised Dealers G. 
 199 of 295 6/2021 
be granted only to stockbrokers, banks and other financial 
institutions....

2. Score: 0.502
   Preview: (F) Technology, media, telecommunications, exploration and other research 
and development companies 
 
(i) Authorised Dealers are advised that unlist...

3. Score: 0.5012
   Preview: Currency and Exchanges M

{'error': None,
 'response': 'Here is a list of Authorised Dealers:\n\n1. ABSA Bank Limited\n2. Bidvest Bank Limited\n3. Citibank N.A., South Africa Branch\n4. FirstRand Bank Limited\n5. HSBC Bank plc – Johannesburg Branch\n6. Investec Bank Limited\n7. Mercantile Bank Limited\n8. Nedbank Limited\n9. Sasfin Bank Limited\n10. Société Générale Johannesburg Branch\n11. Standard Chartered Bank – Johannesburg Branch\n12. The Standard Bank of South Africa Limited\n\nEach of these institutions has specific authorized branches or operational centers listed as well.',
 'sources': [{'node_id': '660991d8-4acf-44cb-aacb-baa6200dbe5d',
   'score': 0.504,
   'text_preview': 'Currency and Exchanges Manual for Authorised Dealers G. \n 199 of 295 6/2021 \nbe granted only to stockbrokers, banks and other financial \ninstitutions....',
   'full_text': 'Currency and Exchanges Manual for Authorised Dealers G. \n 199 of 295 6/2021 \nbe granted only to stockbrokers, banks and other financial \ninstitutions. A

# Simplify/Clarify Model Response

In [30]:
def models_response(user_input):
    """
    Run an example query using the documents and return the result.
    
    Returns:
        dict: Dictionary containing the response text, source information, and metadata
    """
    result = query_documents(user_input)
    if result["error"]:
        print(f"Error: {result['error']}")
    else:
        print(result['response'])
    
    return result


In [31]:
models_response('list Authorised Dealers')

Here is a list of Authorised Dealers:

1. ABSA Bank Limited
2. Access Bank (South Africa) Limited
3. Albaraka Bank Limited
4. Bank of China Johannesburg Branch
5. Bank of Communications Co. Limited Johannesburg Branch
6. Bank of Taiwan South Africa Branch
7. Bidvest Bank Limited
8. Capitec Bank Limited
9. China Construction Bank, Johannesburg Branch
10. Citibank, N.A., South Africa
11. Deutsche Bank AG, Johannesburg Branch
12. Discovery Bank Limited
13. FirstRand Bank Limited
14. Goldman Sachs International Bank, Johannesburg Branch
15. Habib Overseas Bank Limited
16. HBZ Bank Limited
17. HSBC Bank plc – Johannesburg Branch
18. Investec Bank Limited
19. JPMorgan Chase Bank (Johannesburg Branch)
20. Nedbank Limited
21. Sasfin Bank Limited
22. Standard Chartered Bank – Johannesburg Branch
23. State Bank of India
24. The Standard Bank of South Africa Limited

These banks are authorized to act as Authorized Dealers for the purposes of the Regulations in South Africa.


{'error': None,
 'response': 'Here is a list of Authorised Dealers:\n\n1. ABSA Bank Limited\n2. Access Bank (South Africa) Limited\n3. Albaraka Bank Limited\n4. Bank of China Johannesburg Branch\n5. Bank of Communications Co. Limited Johannesburg Branch\n6. Bank of Taiwan South Africa Branch\n7. Bidvest Bank Limited\n8. Capitec Bank Limited\n9. China Construction Bank, Johannesburg Branch\n10. Citibank, N.A., South Africa\n11. Deutsche Bank AG, Johannesburg Branch\n12. Discovery Bank Limited\n13. FirstRand Bank Limited\n14. Goldman Sachs International Bank, Johannesburg Branch\n15. Habib Overseas Bank Limited\n16. HBZ Bank Limited\n17. HSBC Bank plc – Johannesburg Branch\n18. Investec Bank Limited\n19. JPMorgan Chase Bank (Johannesburg Branch)\n20. Nedbank Limited\n21. Sasfin Bank Limited\n22. Standard Chartered Bank – Johannesburg Branch\n23. State Bank of India\n24. The Standard Bank of South Africa Limited\n\nThese banks are authorized to act as Authorized Dealers for the purposes o

In [1]:
models_response('Authorised Dealers in foreign exchange with limited authority') 

NameError: name 'models_response' is not defined